In [1]:
# Road image file path
train_data_dir = "./Covid-19-dataset/train"
test_data_dir = "./Covid-19-dataset/test"
class_list = ["Nomal", "Covid", "Viral Pneumonia"]

In [2]:
# function of set file
class Chest_dataset():
    def __init__(self, data_dir, transformer=None):
        self.data_dir = data_dir
        normals = list_image_files(data_dir, "Normal")
        covids = list_image_files(data_dir, "Covid")
        pneumonia = list_image_files(data_dir, "Viral Pneumonia")
        
        self.files_path = normals + covids + pnrumonias
        self.transformer = transformer
        
    def __len__(self,):
        return len(self.files_path)
    
    def __getitem__(self,):
        image_file = os.path_join(self.data_dir, self.files_path[index])
        image = cv2.imread(image_file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        class_name = self.files_path[index].split(os.sep)[0]
        class_id = class_list.index(class_name)
        
        if self.transformer:
            image = self.transformer(image)
            class_id = torch.Tensor([target]).long()
            
            return image, class_id

In [4]:
train_dset = Chest_dataset(data_dir=train_data_dir)

NameError: name 'list_image_files' is not defined